# Wheel Project

## Setting up the Simulation Environment

In [1]:
from condynsate import Simulator as conSim
from condynsate import __assets__ as assets
import numpy as np

In [2]:
simulator = conSim(keyboard = True,
                   visualization = True,
                   animation = True)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
wheel = simulator.load_urdf(urdf_path = assets['wheel'],
                            position = [0., 0., 0.],
                            fixed = True,
                            update_vis = True)

In [4]:
# Set joint damping
simulator.set_joint_damping(urdf_obj = wheel,
                            joint_name = "ground_to_axle",
                            damping = 0.01)

## Setting up the Animator

In [5]:
plot_1, plot_1_artists = simulator.add_plot(n_artists = 2,
                                            plot_type = 'line',
                                            y_lim = [-0.6283, 6.9115],
                                            title = "Angles vs Time",
                                            x_label = "Time [Seconds]",
                                            y_label = "Angles [Rad]",
                                            color = ["k", "m"], 
                                            line_width = [2.5, 1.5],   
                                            line_style = ["-", "--"],   
                                            label = ["Measured", "Target"], 
                                            h_zero_line = True)

In [6]:
plot_2, plot_2_artists = simulator.add_plot(n_artists = 2,
                                            plot_type = 'bar',
                                            x_lim = [0, 5.0],
                                            title = "Control Gains",
                                            x_label = "Set Value",
                                            color = ["r", "b"],
                                            label = ["Proportional", "Derivative"])

In [7]:
plots = (plot_1, plot_2)
artists = (plot_1_artists, plot_2_artists)

In [8]:
%%capture 
simulator.start_animator()

## Setting up the Simulation Loop

In [9]:
def detect_keypresses(simulator):
    target_iter_val = 0.0
    if simulator.is_pressed('q'): target_iter_val -= 0.02
    if simulator.is_pressed('e'): target_iter_val += 0.02
    P_iter_val = 0.0
    if simulator.is_pressed('f'): P_iter_val -= 0.02
    if simulator.is_pressed('r'): P_iter_val += 0.02
    D_iter_val = 0.0
    if simulator.is_pressed('g'): D_iter_val -= 0.02
    if simulator.is_pressed('t'): D_iter_val += 0.02
    return (target_iter_val, P_iter_val, D_iter_val)

In [10]:
def get_target_error(simulator, wheel, target_angle):
    wheel_state = simulator.get_joint_state(urdf_obj = wheel,
                                            joint_name = "ground_to_axle")
    wheel_angle = wheel_state['position']
    angle_error = target_angle - wheel_angle
    wheel_velocity = wheel_state['velocity']
    velocity_error = 0.0 - wheel_velocity
    return (angle_error, velocity_error)

In [11]:
def set_target_angle(simulator, wheel, error):
    simulator.set_joint_position(urdf_obj = wheel,
                                 joint_name = "wheel_to_target",
                                 position = error[0])

In [12]:
def set_torque(simulator, wheel, error, gains):
    torque = gains[0] * error[0] + gains[1] * error[1]
    simulator.set_joint_torque(urdf_obj = wheel,
                               joint_name = "ground_to_axle",
                               torque = torque)

In [13]:
def update_angle_plot(simulator, plot, artists, target_angle, error):
    time = simulator.time
    wheel_angle = target_angle-error[0]
    simulator.add_line_datum(plot, time, wheel_angle, artist_id=artists[0])
    simulator.add_line_datum(plot, time, target_angle, artist_id=artists[1])

In [14]:
def update_gain_plot(simulator, plot, artists, gains):
    for a,g in zip(artists, gains):
        simulator.set_bar_value(plot, g, artist_id=a)

In [15]:
def step(simulator, plots, artists, target_angle, gains):
    target_iter_val, P_iter_val, D_iter_val = detect_keypresses(simulator)
    target_angle = np.clip(target_angle + target_iter_val, 0.0, 6.2832)
    gains[0] = np.clip(gains[0] + P_iter_val, 0.0, 5.0)
    gains[1] = np.clip(gains[1] + D_iter_val, 0.0, 5.0)

    error = get_target_error(simulator, wheel, target_angle)
    
    set_target_angle(simulator, wheel, error)
    set_torque(simulator, wheel, error, gains)

    update_angle_plot(simulator, plots[0], artists[0], target_angle, error)
    update_gain_plot(simulator, plots[1], artists[1], gains)

    return target_angle, gains

## Running the Simulation Loop

In [16]:
# Set the initial gains and target angle
target_angle = 0.0
gains = [0.0, 0.0]

# Reset before running a simulation loop
simulator.reset()

# Suspend execution until user presses enter key
simulator.await_keypress(key='enter')

# Run the simulation loop
while(not simulator.is_done):
    # Step the sim
    ret_code = simulator.step(real_time = True)

    # If successful step was taken
    if ret_code > 0:
        target_angle, gains = step(simulator, plots, artists, target_angle, gains)
        

PRESS ENTER TO START SIMULATION.
PRESS ESC TO QUIT.
PRESS SPACE TO PAUSE/RESUME SIMULATION.
PRESS BACKSPACE TO RESET SIMULATION.
CONTINUING...
QUITTING...


In [17]:
simulator.terminate_animator()

Termination command detected. Terminating keyboard listener. Goodbye
